In [1]:
# %matplotlib inline
import numpy as np
import os
import snappy
from snappy import Product, ProductIO, HashMap, GPF

# For shapefiles
# import shapefile
# import pygeoif
filename = 'S1A_IW_GRDH_1SDV_20240117T085136_20240117T085201_052144_064D8C_6097.SAFE'
path_to_sentinel_data = "./data/" + filename + ".zip"
product = ProductIO.readProduct(path_to_sentinel_data)

In [2]:
width = product.getSceneRasterWidth()
print("Width: {} px".format(width))
height = product.getSceneRasterHeight()
print("Height: {} px".format(height))
name = product.getName()
print("Name: {}".format(name))
band_names = product.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 25972 px
Height: 16834 px
Name: S1A_IW_GRDH_1SDV_20240117T085136_20240117T085201_052144_064D8C_6097
Band names: Amplitude_VH, Intensity_VH, Amplitude_VV, Intensity_VV


In [3]:
parameters = HashMap()
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
parameters.put('orbitType', 'Sentinel Precise (Auto Download)')
parameters.put('polyDegree', '3')
parameters.put('continueOnFail', 'false')

apply_orbit_file = GPF.createProduct('Apply-Orbit-File', parameters, product)

In [4]:
width = apply_orbit_file.getSceneRasterWidth()
print("Width: {} px".format(width))
height = apply_orbit_file.getSceneRasterHeight()
print("Height: {} px".format(height))
band_names = apply_orbit_file.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 25972 px
Height: 16834 px
Band names: Amplitude_VH, Intensity_VH, Amplitude_VV, Intensity_VV


In [5]:
parameters = HashMap()
parameters.put('outputSigmaBand', True)
parameters.put('sourceBands', 'Intensity_VV,Intensity_VH')
parameters.put('selectedPolarisations', "VV,VH")
# parameters.put('sourceBands', 'Intensity_VH')
# parameters.put('selectedPolarisations', "VH")
parameters.put('outputImageScaleInDb', False)

product_calibrated = GPF.createProduct("Calibration", parameters, apply_orbit_file)

In [6]:
width = product_calibrated.getSceneRasterWidth()
print("Width: {} px".format(width))
height = product_calibrated.getSceneRasterHeight()
print("Height: {} px".format(height))
band_names = product_calibrated.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 25972 px
Height: 16834 px
Band names: Sigma0_VV, Sigma0_VH


In [7]:
filterSizeY = '5'
filterSizeX = '5'
parameters = HashMap()
# parameters.put('sourceBands', 'Sigma0_VV')
parameters.put('filter', 'Lee')
parameters.put('filterSizeX', filterSizeX)
parameters.put('filterSizeY', filterSizeY)
parameters.put('dampingFactor', '2')
parameters.put('estimateENL', 'true')
parameters.put('enl', '1.0')
parameters.put('numLooksStr', '1')
parameters.put('targetWindowSizeStr', '3x3')
parameters.put('sigmaStr', '0.9')
parameters.put('anSize', '50')
speckle_filter = GPF.createProduct('Speckle-Filter', parameters,product_calibrated)

In [8]:
width = speckle_filter.getSceneRasterWidth()
print("Width: {} px".format(width))
height = speckle_filter.getSceneRasterHeight()
print("Height: {} px".format(height))
band_names = speckle_filter.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 25972 px
Height: 16834 px
Band names: Sigma0_VV, Sigma0_VH


In [9]:
parameters = HashMap()
parameters.put('demName', 'SRTM 3Sec')
parameters.put('pixelSpacingInMeter', 10.0)
# parameters.put('sourceBands', 'Sigma0_VV')
speckle_filter_tc = GPF.createProduct("Terrain-Correction", parameters, speckle_filter)

In [10]:
width = speckle_filter_tc.getSceneRasterWidth()
print("Width: {} px".format(width))
height = speckle_filter_tc.getSceneRasterHeight()
print("Height: {} px".format(height))
band_names = speckle_filter_tc.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 29281 px
Height: 22515 px
Band names: Sigma0_VV, Sigma0_VH


In [11]:
ProductIO.writeProduct(speckle_filter_tc, f"./data/final_{filename}", 'BEAM-DIMAP')
os.path.exists(f"./data/final_{filename}.dim")

True